### **Currency Trading Strategy**

In [ ]:
import math
import networkx as nx

##Define currencies and exchange rates.
rates = {
    ("GBP", "EUR"): 1.1989,
    ("GBP", "USD"): 1.2829,
    ("GBP", "CAD"): 1.7008,
    ("EUR", "GBP"): 0.8341,
    ("EUR", "USD"): 1.0373,
    ("EUR", "CAD"): 1.4185,
    ("USD", "GBP"): 0.7795,
    ("USD", "EUR"): 0.9640,
    ("USD", "CAD"): 1.3673,
    ("CAD", "GBP"): 0.5880,
    ("CAD", "EUR"): 0.7050,
    ("CAD", "USD"): 0.7314
}

##Build a graph
G = nx.DiGraph()
for (src, dst), rate in rates.items():
    weight = -math.log(rate)
    G.add_edge(src, dst, rate=rate, weight=weight)

def canonical_cycle(cycle):
    cycle = cycle[:]
    n = len(cycle)
    rotations = [tuple(cycle[i:] + cycle[:i]) for i in range(n)]
    reversed_cycle = list(reversed(cycle))
    rotations += [tuple(reversed_cycle[i:] + reversed_cycle[:i]) for i in range(n)]
    return min(rotations)

##Get all cycles
all_cycles = list(nx.simple_cycles(G))

##Get unique cycles
unique_cycles = set()
for cycle in all_cycles:
    if len(cycle) < 2:
        continue
    canon = canonical_cycle(cycle)
    unique_cycles.add(canon)

print("Unique cycles and their arbitrage analysis:")
for cycle in unique_cycles:
    cycle_complete = list(cycle) + [cycle[0]]
    product = 1
    for i in range(len(cycle_complete) - 1):
        src = cycle_complete[i]
        dst = cycle_complete[i + 1]
        product *= G[src][dst]["rate"]
    print(f"Cycle: {cycle_complete}, Product: {product:.10f}")
    if product > 1:
        print("  --> Arbitrage opportunity detected!")


Unique cycles and their arbitrage analysis:
Cycle: ['EUR', 'GBP', 'USD', 'EUR'], Product: 1.0315444820
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'EUR', 'GBP', 'CAD'], Product: 1.0001392824
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'GBP', 'USD', 'CAD'], Product: 1.0314161920
  --> Arbitrage opportunity detected!
Cycle: ['EUR', 'GBP', 'EUR'], Product: 1.0000024900
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'GBP', 'EUR', 'USD', 'CAD'], Product: 0.9998353280
Cycle: ['GBP', 'USD', 'GBP'], Product: 1.0000205500
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'GBP', 'CAD'], Product: 1.0000704000
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'EUR', 'USD', 'CAD'], Product: 0.9999017045
Cycle: ['EUR', 'USD', 'EUR'], Product: 0.9999572000
Cycle: ['CAD', 'EUR', 'GBP', 'USD', 'CAD'], Product: 1.0314872334
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'EUR', 'CAD'], Product: 1.0000425000
  --> Arbitrage opportunity detected!
Cycle: ['CAD', 'USD', 'C

### **Futures with dividends**

In [1]:
import numpy as np
import pandas as pd

##Dictionary of data
data = {
    "Stock": ["BP Plc", "Shell Plc", "Chevron Corp", "Exxon Mobil Corp"],
    "Current Value (GBX)": [4.49, 27.72, 126.62, 90.53],
    "Dividend (GBX)": [0.63, 2.25, 4.19, 2.62],
    "Forward Strike Price (GBX)": [3.34, 23.96, 121.81, 87.85],
}

r = 0.0492
T = 7 / 12
t_dividends = [2/12, 5/12]


##Functions
def present_value_dividends(dividend, r, t_dividends):
    return sum(dividend * np.exp(-r * t) for t in t_dividends)

def theoretical_forward_price(S0, PV_dividends, r, T):
    return (S0 - PV_dividends) * np.exp(r * T)

df = pd.DataFrame(data)
df["Sum of Present Value Dividends (GBX)"] = df["Dividend (GBX)"].apply(
    lambda D: present_value_dividends(D, r, t_dividends)
)
df["Theoretical Forward Price (GBX)"] = df.apply(
    lambda row: theoretical_forward_price(row["Current Value (GBX)"], row["Sum of Present Value Dividends (GBX)"], r, T),
    axis=1
)

df.to_csv("futures_arbitrage_report.csv", index=False)
df.to_latex("futures_arbitrage_report.tex", index=False)
print(df.to_markdown(index=False))

| Stock            |   Current Value (GBX) |   Dividend (GBX) |   Forward Strike Price (GBX) |   Sum of Present Value Dividends (GBX) |   Theoretical Forward Price (GBX) |
|:-----------------|----------------------:|-----------------:|-----------------------------:|---------------------------------------:|----------------------------------:|
| BP Plc           |                  4.49 |             0.63 |                         3.34 |                                1.24207 |                           3.34249 |
| Shell Plc        |                 27.72 |             2.25 |                        23.96 |                                4.43597 |                          23.962   |
| Chevron Corp     |                126.62 |             4.19 |                       121.81 |                                8.26076 |                         121.805   |
| Exxon Mobil Corp |                 90.53 |             2.62 |                        87.85 |                                5.16544 |     

### **Futures with Cost of Carry**

In [ ]:
import numpy as np
import pandas as pd

##Dictionary of data
data = {
    "Stock": ["Gold", "Silver", "Platinum", "Copper"],
    "Current Value (GBP/Oz)": [2990.26, 33.90, 1069.77, 4.65],
    "Cost of Carry (GBP/Oz)": [125.58, 2.09, 52.33, 0.31],
    "Forward Strike Price (GBP/Oz)": [3217.62, 37.15, 1158.64, 5.12],
}


r = 0.0492
T = 8 / 12
payment_times = [4/12]

def present_value_storage_costs(storage_cost, r, payment_times):
    return sum(storage_cost * np.exp(-r * t) for t in payment_times)

def theoretical_forward_price(S0, storage_costs_PV, r, T):
    return (S0 + storage_costs_PV) * np.exp(r * T)

df = pd.DataFrame(data)
df["Total Storage Cost (GBP/Oz) PV"] = df["Cost of Carry (GBP/Oz)"].apply(
    lambda U: present_value_storage_costs(U, r, payment_times)
)
df["Theoretical Forward Price (GBP/Oz)"] = df.apply(
    lambda row: theoretical_forward_price(row["Current Value (GBP/Oz)"],
                                            row["Total Storage Cost (GBP/Oz) PV"],
                                            r, T),
    axis=1
)
df["Profit (GBP/Oz)"] = df["Theoretical Forward Price (GBP/Oz)"] - df["Forward Strike Price (GBP/Oz)"]
df["Total profit"] = np.floor(10000 / (df["Current Value (GBP/Oz)"]+df['Total Storage Cost (GBP/Oz) PV']))  * df["Profit (GBP/Oz)"]

df.to_csv("futures_carry_report.csv", index=False)
df.to_latex("futures_carry_report.tex", index=False)
print(df.to_markdown(index=False))


| Stock    |   Current Value (GBP/Oz) |   Cost of Carry (GBP/Oz) |   Forward Strike Price (GBP/Oz) |   Total Storage Cost (GBP/Oz) PV |   Theoretical Forward Price (GBP/Oz) |   Profit (GBP/Oz) |   Total profit |
|:---------|-------------------------:|-------------------------:|--------------------------------:|---------------------------------:|-------------------------------------:|------------------:|---------------:|
| Gold     |                  2990.26 |                   125.58 |                         3217.62 |                       123.537    |                           3217.62    |       0.00327301  |     0.00981903 |
| Silver   |                    33.9  |                     2.09 |                           37.15 |                         2.056    |                             37.1549  |       0.00491511  |     1.3664     |
| Platinum |                  1069.77 |                    52.33 |                         1158.64 |                        51.4788   |                 

### **Options Strategy**

In [ ]:
import numpy as np
import pandas as pd

##Dictionary of data
data = {
    "Stock": ["Astrazeneca", "GSK", "J&J", "Pfizer"],
    "Current Price (S)": [123.89, 1548.35, 128.97, 4.63],
    "Call Premium (C)": [713.17, 850.14, 72.91, 0.29],
    "Put Premium (P)": [716.40, 928.88, 84.95, 0.74],
    "Strike Price (X)": [130.82, 1674.46, 141.28, 5.23]
}

df = pd.DataFrame(data)

r = 0.0492
T = 7 / 12
discount_factor = np.exp(-r * T)

def check_arbitrage(row):
    S = row["Current Price (S)"]
    C = row["Call Premium (C)"]
    P = row["Put Premium (P)"]
    X = row["Strike Price (X)"]

    lhs = C + (X * discount_factor)
    rhs = S + P
    discrepancy = lhs - rhs

    profit = abs(discrepancy) * np.exp(r * T)

    return pd.Series([lhs, rhs, discrepancy, profit],
                     index=["LHS (C+Xe^(-rT))", "RHS (S+P))", "Discrepancy", "Profit"])

df = df.join(df.apply(check_arbitrage, axis=1))

df.to_csv("options_report.csv", index=False)
df.to_latex("options_report.tex", index=False)
print(df.to_markdown(index=False))


| Stock       |   Current Price (S) |   Call Premium (C) |   Put Premium (P) |   Strike Price (X) |   LHS (C+Xe^(-rT)) |   RHS (S+P)) |   Discrepancy |     Profit |
|:------------|--------------------:|-------------------:|------------------:|-------------------:|-------------------:|-------------:|--------------:|-----------:|
| Astrazeneca |              123.89 |             713.17 |            716.4  |             130.82 |          840.289   |       840.29 |   -0.00116819 | 0.0012022  |
| GSK         |             1548.35 |             850.14 |            928.88 |            1674.46 |         2477.23    |      2477.23 |   -0.0039343  | 0.00404885 |
| J&J         |              128.97 |              72.91 |             84.95 |             141.28 |          210.193   |       213.92 |   -3.7271     | 3.83562    |
| Pfizer      |                4.63 |               0.29 |              0.74 |               5.23 |            5.37203 |         5.37 |    0.00203249 | 0.00209167 |
